# Sample Code

## 基礎建設

In [16]:
import pandas as pd
import gzip, json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

## 載入資料

In [17]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

--2021-12-26 15:30:00--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15499476 (15M) [application/octet-stream]
Saving to: ‘All_Beauty.csv.1’

All_Beauty.csv.1    100%[===================>]  14.78M  8.81MB/s    in 1.7s    

2021-12-26 15:30:03 (8.81 MB/s) - ‘All_Beauty.csv.1’ saved [15499476/15499476]

--2021-12-26 15:30:03--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10329961 (9.9M) [application/octet-stream]
Saving to: ‘meta_All_Beauty.json.gz.1’

meta_All_Beauty.jso 100%[===================>]   9.85M  6.54MB/s    in 1.5s   

In [18]:
metadata = getDF('/content/meta_All_Beauty.json.gz')
ratings = pd.read_csv('/content/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)

In [19]:
metadata.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Loud 'N Clear Personal Sound Amplifier allows...,,Loud 'N Clear&trade; Personal Sound Amplifier,[],,idea village,[],"2,938,573 in Beauty & Personal Care (",[],{'ASIN: ': '6546546450'},All Beauty,,,,6546546450,[],[]
1,[],,[No7 Lift & Luminate Triple Action Serum 50ml ...,,No7 Lift &amp; Luminate Triple Action Serum 50...,"[B01E7LCSL6, B008X5RVME]",,,[],"872,854 in Beauty & Personal Care (",[],"{'Shipping Weight:': '0.3 ounces (', 'ASIN: ':...",All Beauty,"class=""a-bordered a-horizontal-stripes a-spa...",,$44.99,7178680776,[],[]
2,[],,[No7 Stay Perfect Foundation now stays perfect...,,No7 Stay Perfect Foundation Cool Vanilla by No7,[],,No7,[],"956,696 in Beauty & Personal Care (","[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]","{'Shipping Weight:': '3.5 ounces (', 'ASIN: ':...",All Beauty,,,$28.76,7250468162,[],[]
3,[],,[],,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[B0041PBXX8],,,[],"1,870,258 in Beauty & Personal Care (",[],"{' Item Weight: ': '1.76 ounces', 'Sh...",All Beauty,,,,7367905066,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[Lacto Calamine Skin Balance Daily Nourishing ...,,Lacto Calamine Skin Balance Oil control 120 ml...,[],,Pirmal Healthcare,[],"67,701 in Beauty & Personal Care (","[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...","{'Shipping Weight:': '12 ounces (', 'ASIN: ': ...",All Beauty,,,$12.15,7414204790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [20]:
ratings.head()

,asin,reviewerID,overall,unixReviewTime
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800
2,0143026860,A1572GUYS7DGSR,4.0,1407628800
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200


## 資料整理

In [21]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')

## 資料切分

In [22]:
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())

In [23]:
ratings_trainings

,asin,reviewerID,overall,unixReviewTime,DATE
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000,2015-02-19
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800,2014-12-18
2,0143026860,A1572GUYS7DGSR,4.0,1407628800,2014-08-10
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000,2013-03-11
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200,2011-12-25
...,...,...,...,...,...
371340,B01HJEGTYK,A202DCI7TV1022,1.0,1500508800,2017-07-20
371341,B01HJEGTYK,A3FSOR5IJOFIBE,5.0,1489622400,2017-03-16
371342,B01HJEGTYK,A1B5DK6CTP2P24,5.0,1488326400,2017-03-01
371343,B01HJEGTYK,A23OUYS5IRMJS9,2.0,1487635200,2017-02-21


In [24]:
ratings_testings

,asin,reviewerID,overall,unixReviewTime,DATE
234795,B01CJNZKZK,A38KP6TI3BUIM6,1.0,1536192000,2018-09-06
234796,B01CJNZKZK,A2H7VSYLKGKLLJ,1.0,1535932800,2018-09-03
234797,B01CJNZKZK,A1119JJ37ZLB8R,5.0,1535932800,2018-09-03
240438,B01DKQAXC0,A1FKUBRPLG6Y8I,5.0,1537401600,2018-09-20
240439,B01DKQAXC0,A2WX2YLLHJDLX7,5.0,1537315200,2018-09-19
...,...,...,...,...,...
371154,B01HFWIKH0,A3UAJ5O4VR3IQ3,5.0,1536710400,2018-09-12
371173,B01HGJHL8G,A3254BNN69CL47,5.0,1536105600,2018-09-05
371246,B01HHWQX8G,A32D5I46E15TEC,3.0,1538265600,2018-09-30
371247,B01HHWQX8G,A1S3NMNTSO6GW,4.0,1537142400,2018-09-17


## 產生推薦 random_based

In [25]:
def recommender_random_based(training_data, users=[], k=10):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}
    '''
    random-based
    '''
    ratings_trainings = training_data
    recommendations = {user: ratings_trainings['asin'].sample(n=k).tolist() for user in users}
    return recommendations
ratings_by_user_random = recommender_random_based(ratings_trainings, users)
ratings_by_user_random

{'A100XQFWKQ30O2': ['B000FI4S1E',
  'B00MV76O3M',
  'B00ACHTQQA',
  'B00006L9LC',
  'B002NEL6DG',
  'B0051S94NA',
  'B017VY7XCA',
  'B000GLRREU',
  'B00011QUDE',
  'B00L0JYAZG'],
 'A103T1QOGFCSEH': ['B00KBYFI2A',
  'B0017UATIA',
  'B00SLTYGWU',
  'B0007RB09G',
  'B001AWXBIU',
  'B000FOI48G',
  'B00155Z6V2',
  'B000OQFVLS',
  'B016ZLJINM',
  'B001BOKR3E'],
 'A106UKKSJ2KXPF': ['B012YLE74M',
  'B00CCG7UZS',
  '1620213982',
  'B0139D4WPI',
  'B014VQ890G',
  'B000GLRREU',
  'B000R8Y24A',
  'B006XLW2NS',
  'B00OWRCLL8',
  'B004018VP6'],
 'A10A7GV4D5A11V': ['B01DTFC88E',
  'B01DKQAXC0',
  'B01CDEHN1E',
  'B000WYJTZG',
  'B001F51T6S',
  'B01EAFT79U',
  'B00XPIZUVS',
  'B01EENTV2Q',
  'B00VXEWBJK',
  'B00KHYGOHM'],
 'A1119JJ37ZLB8R': ['B0017TZD7S',
  '1620213982',
  'B001QY8QXM',
  'B003983HRI',
  'B01B3QG1VQ',
  'B01GRAK7F4',
  'B0067F28ZW',
  'B007D1E82U',
  'B00TUAVLSG',
  'B00YQBYO1K'],
 'A113UOOLBSZN52': ['B005Q7EM2O',
  'B01GQ12URM',
  'B000PARERW',
  'B00IAZSG5S',
  'B0091OCA86',
  'B015

## 產生推薦 rule_based

In [26]:
# 沒有購買紀錄的人推薦優質熱門商品
# 1.平均分數>=4
# 2.評分次數>=100
# 3.取上述商品的rank推薦

# 有高評分購買紀錄的人 推薦買過相同商品的人 買過的其他商品
# 1.先濾出該ID評分>=4的商品
# 2.找出買過相同商品的人買過的其他商品
# 3.且平均分數>=4
# 4.取上述商品的rank推薦



In [27]:
# 產生新的 metadata 表
df_asin = metadata['asin']
df_brand = metadata['brand']
df_title = metadata['title']
df_price = metadata['price']
df_rank = metadata['rank'].str.split(" ").str.get(0).str.replace(',','')
df_rank2 = pd.to_numeric(df_rank, errors='coerce')
df_description = metadata['description']
metadata_tmp = pd.concat([df_asin,df_brand,df_title,df_price,df_rank2,df_description],axis='columns')
metadata_new = metadata_tmp.set_index('asin')
metadata_new['overall_mean'] = ratings.groupby(by=['asin']).mean()['overall']
metadata_new['reviews_count'] = ratings.groupby(by=['asin']).size()
metadata_new


,brand,title,price,rank,description,overall_mean,reviews_count
asin,,,,,,,
6546546450,idea village,Loud 'N Clear&trade; Personal Sound Amplifier,,2938573.0,[Loud 'N Clear Personal Sound Amplifier allows...,2.5,2.0
7178680776,,No7 Lift &amp; Luminate Triple Action Serum 50...,$44.99,872854.0,[No7 Lift & Luminate Triple Action Serum 50ml ...,3.0,1.0
7250468162,No7,No7 Stay Perfect Foundation Cool Vanilla by No7,$28.76,956696.0,[No7 Stay Perfect Foundation now stays perfect...,5.0,1.0
7367905066,,Wella Koleston Perfect Hair Colour 44/44 Mediu...,,1870258.0,[],5.0,1.0
7414204790,Pirmal Healthcare,Lacto Calamine Skin Balance Oil control 120 ml...,$12.15,67701.0,[Lacto Calamine Skin Balance Daily Nourishing ...,4.4,15.0
...,...,...,...,...,...,...,...
B01HIWLLUK,,"Barielle Pro Textured Grip Cuticle Nipper, Purple",$9.95,2145325.0,[],5.0,1.0
B01HJ1K3YK,Salon Perfect,(Buy 3 Get 1 Free) Salon Perfect Eye Makeup Co...,,1639713.0,[],1.0,1.0
B01HJ84SGM,,NOW D-Mannose 500 mg - 120 Veg Capsules (Pack ...,$55.63,207410.0,[],5.0,5.0


In [28]:
# 優質熱門商品
popular_products = metadata_new.loc[(metadata_new['overall_mean'] >= 4) & (metadata_new['reviews_count'] >= 100) ].sort_values(by=['rank'])
popular_products.reset_index(inplace=True)
popular_products
fix_recommend_top10=popular_products['asin'].unique()[0:10]

In [29]:
# 過濾出評分表中分數 >= 4 的四星評分表
all_ratings_trainings_4 = ratings_trainings.loc[(ratings_trainings['overall']>=4)]
all_ratings_trainings_4_asin = all_ratings_trainings_4.set_index('asin')
all_ratings_trainings_4_reviewerID = all_ratings_trainings_4.set_index('reviewerID')


In [30]:
# 某使用者A購買過且分數 >= 4 的商品
'''
user_ratings_trainings_4 = ratings_trainings.loc[(ratings_trainings['overall']>=4) & (ratings_trainings['reviewerID']=='A2GJX2KCUSR0EI')]
column_asin = ["asin"]
user_buy_item = pd.DataFrame(user_ratings_trainings_4['asin'].unique(),columns=column_asin)
user_buy_item_asin = user_buy_item.set_index('asin')
user_buy_item_asin
'''


'\nuser_ratings_trainings_4 = ratings_trainings.loc[(ratings_trainings[\'overall\']>=4) & (ratings_trainings[\'reviewerID\']==\'A2GJX2KCUSR0EI\')]\ncolumn_asin = ["asin"]\nuser_buy_item = pd.DataFrame(user_ratings_trainings_4[\'asin\'].unique(),columns=column_asin)\nuser_buy_item_asin = user_buy_item.set_index(\'asin\')\nuser_buy_item_asin\n'

In [31]:
# 承上，交集四星評分表
'''
buy_the_same_items_logs=all_ratings_trainings_4_asin.join(user_buy_item_asin, how='right')
buy_the_same_items_logs
'''


"\nbuy_the_same_items_logs=all_ratings_trainings_4_asin.join(user_buy_item_asin, how='right')\nbuy_the_same_items_logs\n"

In [32]:
# 與某使用者A買過相同商品的人
'''
column_reviewerID = ["reviewerID"]
buy_the_same_items_users = pd.DataFrame(buy_the_same_items_logs['reviewerID'].unique(),columns=column_reviewerID)
buy_the_same_items_users_reviewerID = buy_the_same_items_users.set_index('reviewerID')
buy_the_same_items_users_reviewerID
'''

'\ncolumn_reviewerID = ["reviewerID"]\nbuy_the_same_items_users = pd.DataFrame(buy_the_same_items_logs[\'reviewerID\'].unique(),columns=column_reviewerID)\nbuy_the_same_items_users_reviewerID = buy_the_same_items_users.set_index(\'reviewerID\')\nbuy_the_same_items_users_reviewerID\n'

In [33]:
# 承上，與四星評分表交集
'''
buy_others_items_logs = all_ratings_trainings_4_reviewerID.join(buy_the_same_items_users_reviewerID, how='right')
buy_others_items_logs
'''

"\nbuy_others_items_logs = all_ratings_trainings_4_reviewerID.join(buy_the_same_items_users_reviewerID, how='right')\nbuy_others_items_logs\n"

In [34]:
# 找出買過相同商品的人也買過的商品清單
'''
column_asin = ["asin"]
buy_others_items_list = pd.DataFrame(buy_others_items_logs['asin'].unique(),columns=column_asin)
buy_others_items_list_asin = buy_others_items_list.set_index('asin')
buy_others_items_list_asin
'''

'\ncolumn_asin = ["asin"]\nbuy_others_items_list = pd.DataFrame(buy_others_items_logs[\'asin\'].unique(),columns=column_asin)\nbuy_others_items_list_asin = buy_others_items_list.set_index(\'asin\')\nbuy_others_items_list_asin\n'

In [35]:
# 承上的商品清單交集 metadata
'''
recommend_items = metadata_new.join(buy_others_items_list_asin, how='right')
recommend_items
'''

"\nrecommend_items = metadata_new.join(buy_others_items_list_asin, how='right')\nrecommend_items\n"

In [36]:
# 找出平均分數大於等於4分且評分次數大於等於100次 並用 rank 排序
'''
recommend_items_details = recommend_items.loc[(recommend_items['overall_mean'] >= 4) & (recommend_items['reviews_count'] >= 10)].sort_values(by=['rank'])
recommend_items_details.reset_index(inplace=True)
recommend_items_details
recommend_items_details['asin'].unique()[0:10]
'''



"\nrecommend_items_details = recommend_items.loc[(recommend_items['overall_mean'] >= 4) & (recommend_items['reviews_count'] >= 10)].sort_values(by=['rank'])\nrecommend_items_details.reset_index(inplace=True)\nrecommend_items_details\nrecommend_items_details['asin'].unique()[0:10]\n"

In [37]:
def rule(user,k):
  # 某使用者A購買過且分數 >= 4 的商品
  user_ratings_trainings_4 = ratings_trainings.loc[(ratings_trainings['overall']>=4) & (ratings_trainings['reviewerID']==user)]
  if user_ratings_trainings_4.empty:
    recommend_topk=fix_recommend_top10
  else:
    column_asin = ["asin"]
    user_buy_item = pd.DataFrame(user_ratings_trainings_4['asin'].unique(),columns=column_asin)
    user_buy_item_asin = user_buy_item.set_index('asin')

    # 承上，交集四星評分表
    buy_the_same_items_logs=all_ratings_trainings_4_asin.join(user_buy_item_asin, how='right')
    
    # 與某使用者A買過相同商品的人
    column_reviewerID = ["reviewerID"]
    buy_the_same_items_users = pd.DataFrame(buy_the_same_items_logs['reviewerID'].unique(),columns=column_reviewerID)
    buy_the_same_items_users_reviewerID = buy_the_same_items_users.set_index('reviewerID')

    # 承上，與四星評分表交集
    buy_others_items_logs = all_ratings_trainings_4_reviewerID.join(buy_the_same_items_users_reviewerID, how='right')
    buy_others_items_logs

    # 找出買過相同商品的人也買過的商品清單
    column_asin = ["asin"]
    buy_others_items_list = pd.DataFrame(buy_others_items_logs['asin'].unique(),columns=column_asin)
    buy_others_items_list_asin = buy_others_items_list.set_index('asin')

    # 承上的商品清單交集 metadata
    recommend_items = metadata_new.join(buy_others_items_list_asin, how='right')
    
    # 找出平均分數大於等於4分且評分次數大於等於10次 並用 rank 排序
    recommend_items_details = recommend_items.loc[(recommend_items['overall_mean'] >= 4)& (recommend_items['reviews_count'] >= 100)].sort_values(by=['rank'])
    recommend_items_details.reset_index(inplace=True)
    recommend_items_details
    recommend_topk = recommend_items_details['asin'].unique()[0:k]
    if len(recommend_topk)<k:
      recommend_topk = fix_recommend_top10


  return recommend_topk



In [38]:
def recommender_rule_based(training_data, users=[], k=10):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}

    recommendations = {user: rule(user,k).tolist() for user in users}
    return recommendations


ratings_by_user_rule = recommender_rule_based(ratings_trainings, users)
ratings_by_user_rule

{'A100XQFWKQ30O2': ['B006IB5T4W',
  'B001QY8QXM',
  'B00005JS5C',
  'B002OITLG0',
  'B01DJI7796',
  'B000050FDY',
  'B01C39X6TW',
  'B013XKHA4M',
  'B00X4DKZKU',
  'B000A2LRBY'],
 'A103T1QOGFCSEH': ['B006IB5T4W',
  'B001QY8QXM',
  'B00005JS5C',
  'B002OITLG0',
  'B01DJI7796',
  'B000050FDY',
  'B01C39X6TW',
  'B013XKHA4M',
  'B00X4DKZKU',
  'B000A2LRBY'],
 'A106UKKSJ2KXPF': ['B006IB5T4W',
  'B001QY8QXM',
  'B00005JS5C',
  'B002OITLG0',
  'B01DJI7796',
  'B000050FDY',
  'B01C39X6TW',
  'B013XKHA4M',
  'B00X4DKZKU',
  'B000A2LRBY'],
 'A10A7GV4D5A11V': ['B006IB5T4W',
  'B001QY8QXM',
  'B00005JS5C',
  'B002OITLG0',
  'B01DJI7796',
  'B000050FDY',
  'B01C39X6TW',
  'B013XKHA4M',
  'B00X4DKZKU',
  'B000A2LRBY'],
 'A1119JJ37ZLB8R': ['B006IB5T4W',
  'B001QY8QXM',
  'B00005JS5C',
  'B002OITLG0',
  'B01DJI7796',
  'B000050FDY',
  'B01C39X6TW',
  'B013XKHA4M',
  'B00X4DKZKU',
  'B000A2LRBY'],
 'A113UOOLBSZN52': ['B006IB5T4W',
  'B001QY8QXM',
  'B00005JS5C',
  'B002OITLG0',
  'B01DJI7796',
  'B000

## 結果評估

In [39]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score



In [40]:
evaluate(ratings_testings_by_user, ratings_by_user_random)

0.01694915254237288

In [41]:
evaluate(ratings_testings_by_user, ratings_by_user_rule)

0.011864406779661017